In [179]:
import midi

class Estado:
    track = ''
    instrumento = ''
    tempos = []

class Tempo:
    tick = 0
    ms_per_tick = 0

estado = Estado()
notas_sendo_pressionadas = {}
midi_file = midi.read_midifile("file.mid")
midi_file.make_ticks_abs()


def nota_pressionada(nota):
    pitch = nota.pitch
    notas_sendo_pressionadas[pitch] = nota

def imprimir_informacoes_notas(nota_passada, nota_atual):
    quando_nota_passada_foi_pressionada = converter_tick_para_segundos(nota_passada.tick)
    tempo_atual = converter_tick_para_segundos(nota_atual.tick)
    duracao = tempo_atual - quando_nota_passada_foi_pressionada
    # adaptar o valor da velocidade? velocidade eh a forca que se utiliza determinada nota
    velocidade = nota_atual.data[1]
    canal = nota_atual.channel
    print(tempo_atual)
    # print(f'{quando_nota_passada_foi_pressionada} - {duracao} - {nota_atual.pitch} - {velocidade} - {canal} - {estado.track}-{estado.instrumento}')
    
def nota_levantada(nota):
    pitch = nota.pitch
    nota_pressionada = notas_sendo_pressionadas[pitch]
    del notas_sendo_pressionadas[pitch]
    velocity = nota.data[1]
    imprimir_informacoes_notas(nota_pressionada, nota)

def instrumento_selecionado(evento):
    estado.instrumento = evento.text

def track_selecionado(evento):
    estado.track = evento.text
    
def set_tempo(evento):
    mpqn = evento.get_mpqn()
    mpqn = mpqn / 1000
    resolution = midi_file.resolution
    ms_per_tick = mpqn / resolution
    tempo = Tempo()
    tempo.tick = evento.tick
    tempo.ms_per_tick = ms_per_tick
    estado.tempos.append(tempo)

def converter_tick_para_segundos(tick):
    tempos = estado.tempos
    tempo_total = 0
    ultimo_tempo = tempos[0]
    
    # parte mais complexa
    # em cada bloco de tempo, cada tick se comporta de maneira diferente.
    for tempo in tempos:
        if(tick > tempo.tick):
            print(f'tick > tempo.tick')
            print(f'{tick} > {tempo.tick}')
            tempo_total += tempo.ms_per_tick * (tempo.tick - ultimo_tempo.tick)
        else:
            # cuidad no primeiro caso, que o ultimo tempo, eh simplesmente 0
            print(f'tick < tempo.tick -  {tempo_total}')
            return tempo_total - ultimo_tempo.ms_per_tick * (tick - ultimo_tempo.tick)
        ultimo_tempo = tempo
        
    return tempo_total
    
    
for part in midi_file:
    for event in part:
        if event.name == "Note On":
            nota_pressionada(event)
        elif event.name == "Note Off":
            nota_levantada(event)
        elif event.name == "Track Name":
            track_selecionado(event)
        elif event.name == "Instrument Name":
            instrumento_selecionado(event)
        elif event.name == "Set Tempo":
            set_tempo(event)

tick > tempo.tick
384 > 0
tick < tempo.tick -  0.0
tick > tempo.tick
605 > 0
tick > tempo.tick
605 > 384
tick < tempo.tick -  1313.333
557.4824973958334
tick > tempo.tick
384 > 0
tick < tempo.tick -  0.0
tick > tempo.tick
605 > 0
tick > tempo.tick
605 > 384
tick < tempo.tick -  1313.333
557.4824973958334
tick > tempo.tick
384 > 0
tick < tempo.tick -  0.0
tick > tempo.tick
605 > 0
tick > tempo.tick
605 > 384
tick < tempo.tick -  1313.333
557.4824973958334
tick > tempo.tick
605 > 0
tick > tempo.tick
605 > 384
tick < tempo.tick -  1313.333
tick > tempo.tick
768 > 0
tick > tempo.tick
768 > 384
tick < tempo.tick -  1313.333
0.0
tick > tempo.tick
605 > 0
tick > tempo.tick
605 > 384
tick < tempo.tick -  1313.333
tick > tempo.tick
768 > 0
tick > tempo.tick
768 > 384
tick < tempo.tick -  1313.333
0.0
tick > tempo.tick
605 > 0
tick > tempo.tick
605 > 384
tick < tempo.tick -  1313.333
tick > tempo.tick
768 > 0
tick > tempo.tick
768 > 384
tick < tempo.tick -  1313.333
0.0
tick > tempo.tick
768 > 0